# Photoionization and subsequent cascade decay of neutral neon

In [1]:
using JAC

Atomic cascades usually depend also on the prior excitation or ionization of the atom or ion of interest. Therefore, it is desirable to combine information about the excitation and subsequent decay. While the decay arises due to *internal* interactions among the electrons or the spontaneous photon emission, the excitation or ionization may occur due to photon- or electron-impact processes, whose modelling itself is quite cumbersome.

Accurate photo-excitation studies are difficult since the theoretical (transition) energies are often not accurate enough to decide whether incoming photons are sufficiently resonant to trigger the excitation process. Photoionization is perhaps the simplest process which, apart from embedded resonances, are less sensitive to the details of the incoming beams. However, photoionization formally often results in a quite large number of different excited configurations, although with very different cross sections. Moreover, photoionization is often *tuned* over a range of photon energies with energy- and level-dependent cross sections.

Here, we shall describe the cascade that results from the photoionization and subsequent decay of neutral neon, $ 1s^2 2s^2 2p^6 \to (1s 2s^2 2p^6 + 1s^2 2s 2p^6 + 1s^2 2s^2 2p^5) \to ... \to$ Ne$^{q+} + q\: e^-$. For this photoionization-and-subsequent-decay cascade, it is useful and necessary in JAC to compute the photoionization and the decay cascade separately, and to combine the corresponding transition data later at the time of simulations. For the photoionization, we need to perform a `Cascade.Computation` for the `scheme::Cascade.PhotonIonizationScheme`


In [2]:
? Cascade.PhotonIonizationScheme

`struct  Cascade.PhotonIonizationScheme  <:  Cascade.AbstractCascadeScheme`       ... a struct to represent (and generate) a mean-field orbital basis.

```
+ processes             ::Array{Basics.AbstractProcess,1} 
    ... List of the atomic processes that are supported and should be included into the cascade.
+ maxPhotoElectrons     ::Int64                 
    ... (Maximum) Number of photo-electrons in which the initial-and (photo-ionized) final-state configurations can differ 
        from each other; this affects the number of ionized multiplets in the cascade.
+ photonEnergies        ::Array{Float64,1}      
    ... List of photon energies for which this photo-ionization scheme is to be calculated.
```

---

`Cascade.PhotonIonizationScheme()`  ... constructor for an 'default' instance of a Cascade.PhotonIonizationScheme.


for which we have to specify again the physical processes (Photo) as well as the maximum number of electron; at present, only single-electron photoionization is supported here. We should specify also the photon energies (in a.u.). For this **first photoionization step**, all other input data are similar as before:

In [3]:
name = "Photoionization of neutral neon"
grid = Radial.Grid(false)
wa   = Cascade.Computation(Cascade.Computation(); name=name, nuclearModel=Nuclear.Model(10.), grid=grid, 
                           approach=Cascade.AverageSCA(),
                           scheme=Cascade.PhotonIonizationScheme([Photo()], 1, [30.0, 60.0, 90.0]),
                           initialConfigs=[Configuration("1s^2 2s^2 2p^6")] )

Cascade computation   Photoionization of neutral neon  for a (prior) photo-ionization scheme,  in JAC.Cascade.AverageSCA() approach as well as for Z = 10.0 and initial configurations: 
 1s^2 2s^2 2p^6 ,  
 ... in addition, the following parameters/settings are defined: 
> cascade scheme:           Photo-ionization (scheme) as prior part of an atomic cascade:
processes:                  JAC.Basics.AbstractProcess[Photo()]  
maxPhotoElectrons:          1  
photonEnergies:             [30.0, 60.0, 90.0]  
  
> nuclearModel:             Fermi nuclear model for Z = 10.0 with mass = 20.5, radius R = 2.8580081360657426 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0.  
> grid:                     Radial grid:  rnt = 2.0e-6,  h = 0.05,  hp = 0.02,  NoPoints = 600, orderL = 7,  orderS = 8,  nsL = 92,  nsS = 93,  mesh = JAC.Radial.MeshGL(), ...  
r:    [0.0, 2.1327071094699404e-8, 1.0831512390708573e-7]  ...  [6.162114779810744, 6.1841745613033705, 6.1978155733446

and perform the computation:

In [4]:
setDefaults("print summary: open", "16-cascade-computation-photoionization.sum")
resultsPI = perform(wa; output=true)

setDefaults("print summary: close", "")

Maximum number of SCF iterations = 24 is reached ... computations proceed.
(sym, settings.levelSelectionCI, Basics.selectSymmetry(sym, settings.levelSelectionCI)) = (0 +, Inactive LevelSelection., true)
  reset GBL_Storage_XL_Breit storage ...
>> performCI() ...   0.266731 seconds (356.38 k allocations: 16.896 MiB)
 
* Configurations and levels for all given initial multiplets of the cascade, relative to the lowest:
  
  Configuration: 1s^2 2s^2 2p^6 
  --------------------------------------------
    Level  J Parity          Energy [eV]
  --------------------------------------------
       1     0 +      0.000000000000000e+00 
  --------------------------------------------

* Electron configuration used in the (initial part of the) ionizing cascade:

  Configuration(s) with 10 electrons:
      1s^2 2s^2 2p^6    av. BE = -1968.0  [eV]      (1)

  A total of 1 configuration have been defined for this (initial part of the) ionizing cascade, and selected configurations could be removed he

As before, the results are typical written to some `JLD2` file in order to make use of them, even if the Julia session need to be restarted; here, we simply assign these results to the variable `resultsPI` which we shall use in the subsequent steps. Note that three sets of photoionization (transition) data are generated because of the three given photon energies; in the subsequent simulations, we will need to specify for which photon energy a particular simulation has to be made.

In [5]:
resultsPI

Dict{String,Any} with 5 entries:
  "name"                      => "Photoionization of neutral neon"
  "initial multiplets:"       => Multiplet[name:        initial states  …
  "photo-ionizing line data:" => JAC.Cascade.PhotoIonData[photonEnergy:        …
  "cascade scheme"            => Photo-ionization (scheme) as prior part of an …
  "generated multiplets:"     => Multiplet[name:        noName  …

For the **second decay cascade step**, we can either specify the initial hole configurations explicitly (as in the previous example) or we use of the *generated multiplets* from the `resultsPI`. Here, we still have the freedom to restrict the cascade computation to the $1s 2s^2 2p^6$ multiplet since the other multiplets will be generated automatically. Here, the user need to take care that no *computations are missing* nor that *double computations* are carried out which are later not properly recognized in the simulations. -- In this example, we make use of all three generated multiplets to *start* the cascade and allow up to the emission of three Auger electrons, although such high charge states cannot be reached without additional shake processes: This program should recognize this:

In [6]:
iniMultiplets = resultsPI["generated multiplets:"]
name          = "Neon 1s 2s^2 2p^6 decay cascade"
wb            = Cascade.Computation(Cascade.Computation(); name=name, nuclearModel=Nuclear.Model(10.), 
                grid=grid, approach=Cascade.AverageSCA(),
                scheme=Cascade.StepwiseDecayScheme([Auger(), Radiative()], 3, 0, Shell[], Shell[]),
                initialMultiplets=iniMultiplets )

MethodError: MethodError: no method matching JAC.Cascade.StepwiseDecayScheme(::Array{JAC.Basics.AbstractProcess,1}, ::Int64, ::Int64, ::Array{Shell,1}, ::Array{Shell,1})
Closest candidates are:
  JAC.Cascade.StepwiseDecayScheme(::Any, ::Any, ::Any, ::Any, ::Any, !Matched::Any) at /home/fritzsch/fri/JAC.jl/src/module-Cascade.jl:46
  JAC.Cascade.StepwiseDecayScheme(::Array{JAC.Basics.AbstractProcess,1}, ::Int64, !Matched::Dict{Int64,Float64}, !Matched::Int64, ::Array{Shell,1}, !Matched::Array{Shell,1}) at /home/fritzsch/fri/JAC.jl/src/module-Cascade.jl:46

In [7]:
setDefaults("print summary: open", "16-cascade-computation-decay-cascade.sum")
resultsDecay = perform(wb; output=true)

setDefaults("print summary: close", "")

UndefVarError: UndefVarError: wb not defined

Again, the results are kept here locally in `resultsDecay`

In [8]:
resultsDecay

UndefVarError: UndefVarError: resultsDecay not defined

In the **third simulation step**, we combined these data and apply them in a `Cascade.Simulation`; since all energy levels are numbered in descending energy order, we will usual not know the number of the initial level to provide the correct relative occupation but this can easily sorted out in a second run, once all the relevant energy levels have been tabulated for both cascade computations. Not that this level numbers will usually depend even on the photon energies since different levels will be populated for different incident photon energies. In the input below, we consider the combined cascade for the photon energy 30.0 a.u.:

In [9]:
data = [resultsPI, resultsDecay]
name = "Simulation after photoionization and subsequent cascade decay of neutral neon"
wc   = Cascade.Simulation(Cascade.Simulation(), name=name, 
                          properties=Cascade.AbstractSimulationProperty[Cascade.IonDistribution()],  
                          settings=Cascade.SimulationSettings(0., 0., 0., 0., 30., [(28, 2.0)]), 
                          computationData=data )
perform(wc)

UndefVarError: UndefVarError: resultsDecay not defined

For the photon energy = 30 a.u. $\approx$ 810 eV, only the photoionization of $2s$ and $2p$ electrons are possible in the given approximation and the cascade decay does not lead to a further autoionization but all occupation is left in the single-charged ion.

We can repeat these computations for the other photon energies and find that the vast majority of all photoionized atoms will autoionize, leading to doubly-charged ions, while Ne$^{3+}$ can still not be formed in this approximation.

In [10]:
data = [resultsPI, resultsDecay]
name = "Simulation after photoionization and subsequent cascade decay of neutral neon"
wc   = Cascade.Simulation(Cascade.Simulation(), name=name, 
                          properties=Cascade.AbstractSimulationProperty[Cascade.IonDistribution()],  
                          settings=Cascade.SimulationSettings(0., 0., 0., 0., 60., [(28, 2.0)]), 
                          computationData=data )
perform(wc)

UndefVarError: UndefVarError: resultsDecay not defined

In [11]:
data = [resultsPI, resultsDecay]
name = "Simulation after photoionization and subsequent cascade decay of neutral neon"
wc   = Cascade.Simulation(Cascade.Simulation(), name=name, 
                          properties=Cascade.AbstractSimulationProperty[Cascade.IonDistribution()],  
                          settings=Cascade.SimulationSettings(0., 0., 0., 0., 90., [(28, 2.0)]), 
                          computationData=data )
perform(wc)

UndefVarError: UndefVarError: resultsDecay not defined